In [1]:
import pandas as pd
import numpy as np

In [2]:
PATH = "/Users/fcdevos/yabuk-segment-V 0.3.0.xlsx"

# Raw Data

In [3]:
raw = pd.read_excel(PATH)
raw.head(3)

,Customer ID,Grade,Sub Grade,Employment Length,Home Ownership,Annual Income,Loan Status,Payment Plan,Has Product ~ Car,Has Competitor Product ~ Car,...,Balance - sum,Months on Book - min,Months on Book - mean,Months on Book - max,Remaining Term - min,Remaining Term - mean,Remaining Term - max,Value ~ Past Value,Value ~ Future Value,Value ~ Customer Lifetime Value
0,1,A,A1,1.0,MORTGAGE,34000.0,Fully Paid,n,False,False,...,0.0,4,5.500000,7,29,30.500000,32,3317.473026,17187.410619,20504.883645
1,2,A,A1,1.0,MORTGAGE,36000.0,Fully Paid,n,False,False,...,0.0,5,6.333333,9,27,29.666667,31,2800.068976,15759.501569,18559.570545
2,3,A,A1,1.0,MORTGAGE,42000.0,Charged Off,n,False,False,...,10963.6,3,5.000000,8,28,31.000000,33,-5631.294732,20788.430888,15157.136156


# Product Data

In [5]:
# Get a list of the products
products = [c.replace("Has Product ~ ","") for c in raw.columns if c.startswith("Has Product ~ ")]

In [6]:
# Loop through products so we convert this from wide to long
dfs = []
for product in products[:4]:
    df = raw.loc[:, ["Customer ID", "Has Product ~ {}".format(product), 'Value ~ Customer Lifetime Value']].copy()
    df = df[df['Has Product ~ {}'.format(product)] == True].copy()
    df['Product'] = product
    df = df.drop(columns=['Has Product ~ {}'.format(product)])
    dfs.append(df)
df = pd.concat(dfs)
df.columns = ['Customer ID', 'Value', 'Product']
# Randomly make the Take up true of false
# Generate a column of random trues and falses
df['Take Up'] = np.random.choice([True, False], size=len(df))
df = df[['Customer ID', 'Product', 'Take Up', 'Value']]
df.to_csv("../data/01_raw/product_data.csv", index=False)

In [8]:
df['Product'].value_counts()

Product
Debt Consolidation    14394
Credit Card            7455
Home Improvement       2234
Car                     618
Name: count, dtype: int64

# Customer Data

In [11]:
m = raw[["Has Product ~ {}".format(p) for p in products]].sum(axis=1) > 0
customer_data = raw.loc[:, "Customer ID":"Payment Plan"].copy()
customer_data = customer_data[m].copy()
customer_data.to_csv("../data/01_raw/customer_data.csv", index=False)
customer_data.head(3)

,Customer ID,Grade,Sub Grade,Employment Length,Home Ownership,Annual Income,Loan Status,Payment Plan
0,1,A,A1,1.0,MORTGAGE,34000.0,Fully Paid,n
1,2,A,A1,1.0,MORTGAGE,36000.0,Fully Paid,n
2,3,A,A1,1.0,MORTGAGE,42000.0,Charged Off,n
